In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime #Monday's weekday is 0

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

#import datasets
train = pd.read_csv('../input/train.csv')

In [ ]:
train.head()

1. **Feature Engineering**

In [ ]:
 # Creating three new features: weekday, hour and trip duration and add them to an engineered dataset
train_eng = train
train_eng['pickup_datetime'] = train.pickup_datetime.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
train_eng['dropoff_datetime'] = train.dropoff_datetime.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
train_eng['pickup_weekday'] = train_eng.pickup_datetime.apply(datetime.weekday)
train_eng['pickup_hour'] = train_eng.pickup_datetime.apply(lambda x: x.hour)
train_eng['trip_duration'] = train_eng['dropoff_datetime'] - train_eng['pickup_datetime'] 
train_eng['trip_duration_in_seconds'] = train_eng['trip_duration'].apply(lambda x : x.seconds)

In [ ]:
train_eng.describe()

In [ ]:
train_eng[train_eng['trip_duration_in_seconds']<3600*1].trip_duration_in_seconds.hist()